In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 211 kB 4.2 MB/s 
     |████████████████████████████████| 93 kB 1.9 MB/s 
     |████████████████████████████████| 81 kB 9.3 MB/s 
     |████████████████████████████████| 7.4 MB 6.4 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13553 sha256=e555944822a493e9bd69232e5c4d708c0c3d172ad4cfa44b66ed6d64a6706252
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3357 sha256=43a5c9a3e8ee57c26ec0322cfea35af9ae3028b65b5a9fb58ef1435764cfa5b6
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398404 sha256=bc944cf85e0d05ef54efafbe6f3e6043b85c58381ffc1bc69a

In [6]:
import pandas as pd
import numpy as np
import newspaper

import itertools
import requests
import os.path
import string

from bs4 import BeautifulSoup
from datetime import datetime, date
from google.colab import drive
from newspaper import Article

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
def get_all_url(in_df, base_url, news_cat):
    
    all_urls = list(in_df['article_link'])
    categories = list(in_df['category'])
    all_dates = list(in_df['upload_date'])
    all_text = list(in_df['original_text'])

    print(f'\nPrevious data: {len(in_df)}')

    for _category in news_cat:
        # get the returned webpage
        page_url = base_url+_category
        #print(f'PageURl= {page_url}')

        page = requests.get(page_url)
        soup = BeautifulSoup(page.content, 'html.parser')

        _urls = soup.find_all('a')
    
        for _url in _urls:
            _temp_url = _url.get('href')

            #print(_temp_url)

            # check duplicacy
            if _temp_url not in all_urls:
                # get only the urls related with this sub categories
                if str(_temp_url).startswith(page_url):
                    url_arr = _temp_url.replace(page_url, '')

                    #print(url_arr)

                    if len(url_arr.strip()) > 1:
                        #print(_temp_url)
                        all_urls.append(_temp_url)
                        categories.append(_category)
                        all_dates.append('')
                        all_text.append('')

    return pd.DataFrame({'article_link': all_urls, 'upload_date': all_dates, 'original_text': all_text,  'category': categories} )

In [9]:
# define base url and top categories
base_url = 'https://bdnews24.com/'
news_cat = ['politics', 'business', 'economy', 'science', 'technology', 'cricket', 'sport']

try:
    # load previous data file if exist
    df = pd.read_csv('/content/drive/MyDrive/datasets/news_paper_urls.csv')
    print('Get file location')
except FileNotFoundError as e: 
    # initialize the data frame
    print(f'File read error')
    df = pd.DataFrame(columns=['article_link', 'upload_date', 'original_text', 'category'])

df = get_all_url(df, base_url, news_cat)

print(f'Total URL get: {len(df)}')

Get file location

Previous data: 272
Total URL get: 327


In [10]:
df.head()

,article_link,upload_date,original_text,category
0,https://bdnews24.com/politics/2022/07/21/uk-le...,21/07/2022,UK leadership candidate Truss leads rival Suna...,politics
1,https://bdnews24.com/politics/2022/07/20/hasta...,20/07/2022,Johnson was forced to announce his resignation...,politics
2,https://bdnews24.com/politics/2022/07/20/what-...,20/07/2022,Russian President Vladimir Putin used a rare f...,politics
3,https://bdnews24.com/politics/2022/07/19/race-...,19/07/2022,Since Johnson said he would resign earlier thi...,politics
4,https://bdnews24.com/politics/2022/07/19/incre...,19/07/2022,Sunak got 115 votes in the third ballot of Con...,politics


In [11]:
df.tail()

,article_link,upload_date,original_text,category
322,https://bdnews24.com/sport/mane-de-ligt-score-...,,,sport
323,https://bdnews24.com/sport/covid-puts-a-stop-t...,,,sport
324,https://bdnews24.com/sport/ronaldo-not-for-sal...,,,sport
325,https://bdnews24.com/sport/farah-says-he-was-v...,,,sport
326,https://bdnews24.com/sport/fulham-sign-midfiel...,,,sport


In [12]:
#merge categories
def merge_categories(x):
    if x == 'cricket':
        return 'sport'
    elif x == 'economy':
        return 'business'
    elif x == 'science' or x == 'technology':
        return 'science_technology'
    return x

df['category'] = df['category'].apply(merge_categories)

In [13]:
# get all categories
set(df.category)

{'business', 'politics', 'science_technology', 'sport'}

In [14]:
total_article = len(df)

In [15]:
# drop dplicate urls
df = df.drop_duplicates()
print(f'Number of droped urls: {total_article-len(df)}')

Number of droped urls: 0


In [16]:
#df.to_csv('/content/drive/MyDrive/datasets/news_paper_urls.csv', index=False)

In [17]:
for index, row in df.iterrows():

    if len(row['original_text']) == 0:
        # get the article text from the url
        article = Article(row['article_link'])
        article.download()
        article.parse()

        date = '' 

        try:
            date = (article.publish_date).strftime("%d/%m/%Y")
        except: 
            date = ''

        df.at[index, 'upload_date'] = date
        df.at[index, 'original_text'] = article.text

In [18]:
df.head()

,article_link,upload_date,original_text,category
0,https://bdnews24.com/politics/2022/07/21/uk-le...,21/07/2022,UK leadership candidate Truss leads rival Suna...,politics
1,https://bdnews24.com/politics/2022/07/20/hasta...,20/07/2022,Johnson was forced to announce his resignation...,politics
2,https://bdnews24.com/politics/2022/07/20/what-...,20/07/2022,Russian President Vladimir Putin used a rare f...,politics
3,https://bdnews24.com/politics/2022/07/19/race-...,19/07/2022,Since Johnson said he would resign earlier thi...,politics
4,https://bdnews24.com/politics/2022/07/19/incre...,19/07/2022,Sunak got 115 votes in the third ballot of Con...,politics


In [19]:
#read input data
#df = pd.read_csv('/content/drive/MyDrive/datasets/all_input_data.csv')
#print(f'Total number off data points: {len(df)}')

In [20]:
# cleaning the data

processed_text = []

original_text_list = df['original_text']

for index in range(len(original_text_list)):

    text = original_text_list[index]
    # remove new line
    text = text.replace('\n', ' ').strip()

    #remove puncuations
    text = text.translate(str.maketrans('', '', string.punctuation))

    # set lower case
    text = text.lower()

    text_tokens = word_tokenize(text)

    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('english')]

    text = ' '.join(tokens_without_sw)

    processed_text.append(text.strip())
    #numeric_category.append(category_ids.index(row['category']))

    if index%15 == 0:
        print(f'Total Processed complete {round(((index+1)/len(df) * 100),2)} %')
print(f'Total Processed 100%')

Total Processed complete 0.31 %
Total Processed complete 4.89 %
Total Processed complete 9.48 %
Total Processed complete 14.07 %
Total Processed complete 18.65 %
Total Processed complete 23.24 %
Total Processed complete 27.83 %
Total Processed complete 32.42 %
Total Processed complete 37.0 %
Total Processed complete 41.59 %
Total Processed complete 46.18 %
Total Processed complete 50.76 %
Total Processed complete 55.35 %
Total Processed complete 59.94 %
Total Processed complete 64.53 %
Total Processed complete 69.11 %
Total Processed complete 73.7 %
Total Processed complete 78.29 %
Total Processed complete 82.87 %
Total Processed complete 87.46 %
Total Processed complete 92.05 %
Total Processed complete 96.64 %
Total Processed 100%


In [21]:
# add one new column for processed_data
df.insert(df.columns.get_loc("original_text")+1, "processed_data", processed_text)
df.head()

,article_link,upload_date,original_text,processed_data,category
0,https://bdnews24.com/politics/2022/07/21/uk-le...,21/07/2022,UK leadership candidate Truss leads rival Suna...,uk leadership candidate truss leads rival suna...,politics
1,https://bdnews24.com/politics/2022/07/20/hasta...,20/07/2022,Johnson was forced to announce his resignation...,johnson forced announce resignation earlier mo...,politics
2,https://bdnews24.com/politics/2022/07/20/what-...,20/07/2022,Russian President Vladimir Putin used a rare f...,russian president vladimir putin used rare for...,politics
3,https://bdnews24.com/politics/2022/07/19/race-...,19/07/2022,Since Johnson said he would resign earlier thi...,since johnson said would resign earlier month ...,politics
4,https://bdnews24.com/politics/2022/07/19/incre...,19/07/2022,Sunak got 115 votes in the third ballot of Con...,sunak got 115 votes third ballot conservative ...,politics


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df['processed_data'], df['category'], test_size=0.20, random_state=42)

print(f'Shape of X_train= {X_train.shape}, X_test= {X_test.shape}, y_train= {y_train.shape}, y_test= {y_test.shape}')

Shape of X_train= (261,), X_test= (66,), y_train= (261,), y_test= (66,)


In [24]:
# making count vectors
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(261, 11965)

In [25]:
# making tf-idf vectors
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(261, 11965)

In [26]:
# model training and prediction

# Used naïve Bayes classifier
clf = MultinomialNB().fit(X_train_tfidf, y_train)

X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

predicted = clf.predict(X_test_tfidf)

In [27]:
# model evaluation

print(f'Model accuracy is {accuracy_score(y_test, predicted)}\n\n')

d_report = classification_report(y_test, predicted)
print(f'Classification report: \n{d_report}')

Model accuracy is 0.8333333333333334


Classification report: 
                    precision    recall  f1-score   support

          business       0.75      1.00      0.86        21
          politics       1.00      0.44      0.62         9
science_technology       0.94      0.73      0.82        22
             sport       0.82      1.00      0.90        14

          accuracy                           0.83        66
         macro avg       0.88      0.79      0.80        66
      weighted avg       0.86      0.83      0.82        66



In [28]:
df.to_csv('/content/drive/MyDrive/datasets/all_input_data.csv', index=False)